In [1]:
# by running this notebook with raw csv file database.csv you will get file with floats
# and proper dates which is called final.csv and which has New_building, quarter and city
#columns 

In [2]:
import pandas as pd
import numpy as np
import warnings
import ast
from tqdm import tqdm
from datetime import datetime
warnings.filterwarnings('ignore')

In [3]:
# import plotly.offline as pyo
# import plotly.graph_objs as go
tqdm.pandas(desc="my bar!")

In [4]:
flats = pd.read_csv('database.csv',sep=',',dtype='unicode')

делаю числом все, что может быть числом

In [5]:
def string_to_obj(x):
    try:
        tmp = ast.literal_eval(x)
    except:
        tmp = x
    return tmp

In [6]:
flats.keys()

Index(['Building_year', 'Number_of_rooms', '_id', 'active', 'address',
       'bathroom_number', 'bathroom_separated', 'ceiling_type',
       'central_heating', 'cian_id', 'date_of_adding_to_db', 'date_of_place',
       'emergency_condition', 'house_type', 'housing_complex', 'id',
       'kitchen_area', 'latitude', 'living_area', 'longitude',
       'passengers_elevator_number', 'pic_urls', 'porch_num',
       'price_per_house_in_dst', 'price_per_house_in_dst_dynamics',
       'price_per_meter_in_dst', 'price_per_meter_in_dst_dynamics',
       'price_per_sq_meter', 'price_range', 'purchase_dynamics',
       'purchase_price', 'rent_dynamics', 'rent_dynamics_in_dst', 'rent_price',
       'rent_price_in_dst', 'room1_square', 'room2_square', 'room3_square',
       'seen_as_old', 'service_elevator_number', 'storey_number', 'total_area',
       'total_number_views', 'total_price', 'type_of_flat', 'visitors',
       'whole_storey_number', 'windows_to_street', 'сeiling_height',
       'trade_t

In [6]:
flats = flats.progress_applymap(string_to_obj)

my bar!: 100%|██████████████████████████████████████████████████████████| 22191950/22191950 [04:05<00:00, 90409.48it/s]


In [7]:
flats = flats.loc[flats.date_of_adding_to_db.apply(type) != list]

In [8]:
def make_new_buildings(x):
#     print(x.Building_year)
#     print(x.Building_year.isinstance(list))
    if type(x.Building_year) is list and len(x.Building_year)==2:
        lst = x.Building_year
        x['New_building'] = True
        x.Building_year = lst[-1]
        x['quarter'] = lst[0]
    else:
        x['New_building'] = False
        x['quarter'] = None
    return x#.New_building, x.Building_year, x.quarter       

In [9]:
flats  = flats.progress_apply(make_new_buildings, axis=1)

my bar!: 100%|████████████████████████████████████████████████████████████████| 443794/443794 [24:05<00:00, 306.96it/s]


In [10]:
def make_float(x):
    try:
        tmp = float(x)
    except:
        tmp = x
    return tmp

In [11]:
def str_to_date(x):
    if x is not np.nan:
        try: 
            return pd.to_datetime(x.date_of_adding_to_db, format="%Y-%m-%d")
        except: 
            return pd.to_datetime(x.date_of_adding_to_db, format="%d.%m.%Y")
    else:
        return np.nan

In [12]:
flats['date_of_adding_to_db'] = flats.progress_apply(str_to_date, axis=1)

my bar!: 100%|███████████████████████████████████████████████████████████████| 443794/443794 [01:52<00:00, 3932.97it/s]


In [13]:
# flats.date_of_adding_to_db = flats.date_of_adding_to_db.progress_apply(lambda x: datetime.strptime(x, '%Y-%m-%d') if x is not np.nan else np.nan)

In [14]:
flats.date_of_adding_to_db.unique()

array(['2020-02-26T00:00:00.000000000', '2020-02-27T00:00:00.000000000',
       '2020-02-28T00:00:00.000000000', '2020-02-29T00:00:00.000000000',
       '2020-03-01T00:00:00.000000000', '2020-03-02T00:00:00.000000000',
       '2020-03-03T00:00:00.000000000', '2020-03-04T00:00:00.000000000',
       '2020-03-05T00:00:00.000000000', '2020-03-06T00:00:00.000000000',
       '2020-03-07T00:00:00.000000000', '2020-03-08T00:00:00.000000000',
       '2020-03-09T00:00:00.000000000', '2020-03-10T00:00:00.000000000',
       '2020-03-11T00:00:00.000000000', '2020-03-12T00:00:00.000000000',
       '2020-03-13T00:00:00.000000000', '2020-03-14T00:00:00.000000000',
       '2020-03-15T00:00:00.000000000', '2020-03-18T00:00:00.000000000',
       '2020-03-19T00:00:00.000000000', '2020-03-20T00:00:00.000000000',
       '2020-03-21T00:00:00.000000000', '2020-03-22T00:00:00.000000000',
       '2020-03-23T00:00:00.000000000', '2020-03-24T00:00:00.000000000',
       '2020-03-26T00:00:00.000000000', '2020-03-27

In [15]:
def make_float_or_nan(x):
    try:
        tmp = float(x)
    except:
        tmp = np.nan
    return tmp

In [16]:
flats.total_price = flats.total_price.apply(make_float_or_nan)


In [17]:
flats.groupby('Number_of_rooms')['id'].count().sort_values()

Number_of_rooms
5.0      1164
4.0      7271
0.8     14990
3.0     45345
2.0    110723
1.0    112427
Name: id, dtype: int64

In [18]:
flats  = flats.progress_applymap(make_float)

my bar!: 100%|█████████████████████████████████████████████████████████| 23077288/23077288 [00:55<00:00, 413924.39it/s]


In [19]:
flats['city'] = flats.address.apply(lambda x: x[0] if x is not None and x is not np.nan else None)
flats['address'] = flats.address.apply(lambda x: "".join(x) if x is not None and x is not np.nan else None)

In [20]:
flats.head()

,Building_year,Number_of_rooms,_id,active,address,bathroom_number,bathroom_separated,ceiling_type,central_heating,cian_id,...,total_price,type_of_flat,visitors,whole_storey_number,windows_to_street,сeiling_height,trade_type,New_building,quarter,city
13,NaN,NaN,ObjectId(5e56d3f574c444141af62f73),true,None,NaN,NaN,NaN,NaN,225950180.0,...,NaN,NaN,NaN,"[null,null]",NaN,NaN,NaN,0.0,NaN,None
16,NaN,NaN,ObjectId(5e56d3fe0e5436f6f7f62f75),true,None,NaN,NaN,NaN,NaN,225950184.0,...,NaN,NaN,NaN,"[null,null]",NaN,NaN,NaN,0.0,NaN,None
21,NaN,NaN,ObjectId(5e56d421a04896a979f62f75),true,None,NaN,NaN,NaN,NaN,225950189.0,...,NaN,NaN,NaN,3,NaN,NaN,NaN,0.0,NaN,None
25,NaN,NaN,ObjectId(5e56d435baafcdff7ef62f79),true,None,NaN,NaN,NaN,NaN,225950195.0,...,NaN,NaN,NaN,"[null,null]",NaN,NaN,NaN,0.0,NaN,None
27,NaN,NaN,ObjectId(5e56d446baafcdff7ef62f7b),true,None,NaN,NaN,NaN,NaN,225950197.0,...,NaN,NaN,NaN,2,NaN,NaN,sale,0.0,NaN,None


In [21]:
flats.to_csv('final.csv')